In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")

import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns
import pyaldata as pyal

from tools.dsp.preprocessing import preprocess
import tools.viz.rasters as rt
from tools.params import Params
from tools.viz.dimensionality import plot_VAF
from tools.dataTools import get_data_array
from tools.viz.rasters import plot_heatmap_raster

from tools.dimensionality.participation import participation_ratio, pca_pr
from tools.params import colors
from tools.dimensionality.cca import canoncorr



In [4]:
# TODO: add example data to the repo and run on that
data_dir = "/data/bnd-data/raw/M051/M051_2025_02_05_16_00"
fname = os.path.join(data_dir, "M051_2025_02_05_16_00_pyaldata.mat")

In [5]:
# load TrialData .mat file into a DataFrame
df = pyal.mat2dataframe(fname, shift_idx_fields=True)

field values_before_camera_trigger could not be converted to int.
field idx_before_camera_trigger could not be converted to int.
field idx_cursor_update could not be converted to int.
array field MOp_KSLabel could not be converted to int.
array field CP_KSLabel could not be converted to int.
array field all_KSLabel could not be converted to int.


In [6]:
df.head()

,animal,session,trial_id,trial_name,trial_length,bin_size,idx_trial_start,idx_trial_end,idx_CPI,values_before_camera_trigger,...,MOp_KSLabel,MOp_spikes,CP_chan_best,CP_unit_guide,CP_KSLabel,CP_spikes,all_chan_best,all_unit_guide,all_KSLabel,all_spikes
0,M051,M051_2025_02_05_16_00,0,trial,29756,0.01,-1,29754,[],47.0,...,"[good, good, good, good, good, good, good, goo...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 2, 2, 5, 5, 6, 7, 7, 8, 8, 8, 8, 10, 11, 1...","[[1, 1], [2, 1], [2, 2], [5, 1], [5, 2], [6, 1...","[good, mua, good, good, good, good, mua, mua, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[272, 273, 273, 274, 275, 276, 277, 277, 279, ...","[[272, 1], [273, 1], [273, 2], [274, 1], [275,...","[mua, mua, mua, mua, good, good, good, mua, mu...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]..."
1,M051,M051_2025_02_05_16_00,1,cursor_match,13,0.01,29754,29766,[],[],...,"[good, good, good, good, good, good, good, goo...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 2, 2, 5, 5, 6, 7, 7, 8, 8, 8, 8, 10, 11, 1...","[[1, 1], [2, 1], [2, 2], [5, 1], [5, 2], [6, 1...","[good, mua, good, good, good, good, mua, mua, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[272, 273, 273, 274, 275, 276, 277, 277, 279, ...","[[272, 1], [273, 1], [273, 2], [274, 1], [275,...","[mua, mua, mua, mua, good, good, good, mua, mu...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]..."
2,M051,M051_2025_02_05_16_00,2,trial,1290,0.01,29766,31055,[],[],...,"[good, good, good, good, good, good, good, goo...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 2, 2, 5, 5, 6, 7, 7, 8, 8, 8, 8, 10, 11, 1...","[[1, 1], [2, 1], [2, 2], [5, 1], [5, 2], [6, 1...","[good, mua, good, good, good, good, mua, mua, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[272, 273, 273, 274, 275, 276, 277, 277, 279, ...","[[272, 1], [273, 1], [273, 2], [274, 1], [275,...","[mua, mua, mua, mua, good, good, good, mua, mu...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]..."
3,M051,M051_2025_02_05_16_00,3,cursor_match,9,0.01,31055,31063,[],[],...,"[good, good, good, good, good, good, good, goo...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 2, 2, 5, 5, 6, 7, 7, 8, 8, 8, 8, 10, 11, 1...","[[1, 1], [2, 1], [2, 2], [5, 1], [5, 2], [6, 1...","[good, mua, good, good, good, good, mua, mua, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[272, 273, 273, 274, 275, 276, 277, 277, 279, ...","[[272, 1], [273, 1], [273, 2], [274, 1], [275,...","[mua, mua, mua, mua, good, good, good, mua, mu...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]..."
4,M051,M051_2025_02_05_16_00,4,trial,241,0.01,31063,31303,[],[],...,"[good, good, good, good, good, good, good, goo...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 2, 2, 5, 5, 6, 7, 7, 8, 8, 8, 8, 10, 11, 1...","[[1, 1], [2, 1], [2, 2], [5, 1], [5, 2], [6, 1...","[good, mua, good, good, good, good, mua, mua, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[272, 273, 273, 274, 275, 276, 277, 277, 279, ...","[[272, 1], [273, 1], [273, 2], [274, 1], [275,...","[mua, mua, mua, mua, good, good, good, mua, mu...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]..."


In [10]:
np.all(df.CP_spikes[0] == 0)

True